## Ingesting PDF

In [5]:
%pip install --upgrade pip 

  Using cached pip-24.1.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.1.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install unstructured langchain
%pip install "unstructured[all-docs]"
%pip install langchain_community 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: c:\programs\tasks\image chatbot (task2)\venv\Scripts\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: c:\programs\tasks\image chatbot (task2)\venv\Scripts\python.exe -m pip install --upgrade pip


  Using cached langchain_community-0.2.6-py3-none-any.whl.metadata (2.5 kB)
Using cached langchain_community-0.2.6-py3-none-any.whl (2.2 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: c:\programs\tasks\image chatbot (task2)\venv\Scripts\python.exe -m pip install --upgrade pip


In [6]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [7]:
local_path = "iata 1.pdf"

# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

c:\programs\tasks\image chatbot (task2)\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Preview first page
data[0].page_content

"Air Passenger Market Analysis December 2022 The year ends on a strong note for the global industry • In 2022, air passenger traffic gained momentum globally and recovered substantially from 41.7% of 2019 revenue\n\npassenger-kilometers (RPKs) in 2021 to 68.5% in 2022.\n\nDomestic RPKs recovered to 79.6% of pre-pandemic passenger traffic in 2022 and grew 10.9% year-on-year (YoY) from 2021 levels. International RPKs recovered to 62.2% of 2019 traffic and grew 152.7% YoY from 2021 levels. • Industry-wide RPKs increased by 39.7% YoY in December and reached 76.9% of pre-pandemic levels for the same month. Compared to December 2021, domestic passenger traffic grew 2.6% while international traffic grew 80.2%. • Monitored domestic markets continued to show resilience and steady traffic levels. International passenger traffic within and between the Asia Pacific region and the rest of the world also continued to show positive trends.\n\nTotal domestic ticket sales have seen an uptick over the m

## Vector Embeddings

In [9]:
!ollama pull nomic-embed-text

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [10]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED               
nomic-embed-text:latest	0a109f422b47	274 MB	Less than a second ago	
llama3:latest          	365c0bd3c000	4.7 GB	2 days ago            	
mistral:latest         	2ae6f6dd7a3d	4.1 GB	2 days ago            	


In [11]:
%pip install --q chromadb
%pip install --q langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [13]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=750, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [14]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="chat-bot"
)

OllamaEmbeddings: 100%|██████████| 29/29 [01:49<00:00,  3.77s/it]


## Retrieval And Question-Answering

In [15]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [16]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [17]:
#Defining the prompt-template for question-answering
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [18]:
#Configuring a retriever for Retrieval-Augmented Generation (RAG).
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [19]:
# Defining a processing chain for question answering
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [20]:
#Ask questions about the pdf to the model
chain.invoke(input(""))

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.23s/it]


' The context provided is about the Air Passenger Monthly Analysis for December 2022, specifically focusing on the global air passenger market. It discusses changes in load factors, RPKs (Revenue Passenger Kilometers), and ticket sales for both international and domestic markets. It also mentions that the global air passenger traffic recovered substantially throughout 2022 compared to 2019 levels, and that the recovery trends for traffic in the Premium and Economy cabin classes remain broadly aligned. The document is based on data from IATA (International Air Transport Association).'

In [23]:
#ask questions about the pdf to the model
chain.invoke("Which year and month, international ticket sell in higher than domestic sales?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.18s/it]


' The provided context does not explicitly state that there is a specific year and month when international ticket sales exceed domestic sales. However, it does mention that international ticket sales have "caught up to" and maintained their recovery with domestic ticket sales as of the most recent data (January 2023). This suggests that at the very least, in January 2023, international ticket sales are equal to or higher than domestic sales. For specific instances where international ticket sales surpassed domestic sales, additional context or a more detailed comparison would be required.'

In [24]:
chain.invoke("In international RPK's which region has maximum deviation in 2019?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.15s/it]


' The region with the maximum deviation (furthest from pre-pandemic levels) in international RPKs, according to the provided context, is Africa. In December 2019, international RPKs for Africa were at 89.2%, while in December 2022 they were 71.7%. This represents a 14.2% deviation from pre-pandemic levels.'

In [25]:
chain.invoke("In international RPK's which region has minimum deviation in 2019?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.26s/it]


' Based on the provided context, it is not explicitly stated which region had the minimum deviation in 2019 for international RPKs. However, it can be inferred that the regions closest to their 2019 levels are Asia (44.1%) and Europe (14.5%). To definitively answer this question, we would need more specific data comparing the deviation percentages among all regions.'

In [26]:
#  if you want to chat with new pdfs, delete the old vertor_db collections
vector_db.delete_collection()